In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('../data/Train_dataset.csv')
train.head(1)

,Registration Number,Annual Turnover,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [4]:
from auto_ml_kinder import pre_processing as pp

In [5]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [6]:
train.head(1)

,registration_number,annual_turnover,cuisine,city,restaurant_location,opening_day_of_restaurant,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,...,overall_restaurant_rating,live_music_rating,comedy_gigs_rating,value_deals_rating,live_sports_rating,ambience,lively,service,comfortablility,privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [7]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.city.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='endorsed_by',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.endorsed_by.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_theme',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_theme.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cuisine',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
    )

numeric_col_changer:list[pp.PreNumericColDataChangeConfig] = []
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='facebook_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='instagram_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='overall_restaurant_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='live_music_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='ambience',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='resturant_tier',data_type=int))

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column='annual_turnover'
)

In [8]:
pre_processed_df = pp.process(train,model_config)

Dropping columns whose values are nullable greater than 0.7
Removing unusable values like " " and "?"
Filling NA values with median and mode for numeria and non-numeric columns.
Performing IQR based outliers cleaning on target. annual_turnover
Changing datatype of numeric columns.
Performing label encoding on columns provided.
Encoding using dummies.


In [26]:
from auto_ml_kinder import model_training_data_prep as dp
from auto_ml_kinder import model_training_helper as mth
from auto_ml_kinder import model_list_helper as mlh
from auto_ml_kinder.classes.ModelTrainingData import ModelTrainingData
from auto_ml_kinder.classes.ModelTrainer import ModelTrainer
from auto_ml_kinder.classes.ModelAndParamRegression import ModelAndParamRegression
from auto_ml_kinder.classes.ScalerType import ScalerType as st

In [23]:
data = ModelTrainingData(pre_processed_df, st.QUANTILE_TRANSFORMER, .90,use_pca=False,use_polynomials=False,use_feature_selection=False,create_clustering_feature_and_no_of_clusters=(True,5))

Grid tuning in progress for KMeans...
Grid tuning in progress for Agglomerative Clustering...
Mean Shift clustering does not require hyperparameter tuning.
DBSCAN clustering does not require hyperparameter tuning.
Birch clustering does not require hyperparameter tuning.
Affinity Propagation clustering does not require hyperparameter tuning.
Grid tuning in progress for Mini-Batch KMeans...
Mean score for kmeans is: 0.3138318006628074
Mean score for agglomerative is: 0.28354878064080197
Mean score for mean_shift is: 0.7060275066147514
Algorithm dbscan produced an invalid number of clusters: 1
Mean score for birch is: 0.6109554600403464
Mean score for affinity_propagation is: 0.07899823627482828
Mean score for mini_batch_kmeans is: 0.20881899810622012
Best algorithm: mean_shift with a score of: 0.7060275066147514
Mean score of all clusters is: 0.6441476298784594
Total columns being used after all data transformations: 29


In [28]:
trainer = ModelTrainer(data=data)

In [29]:
trainer.perform_operation_regression(exclude_models=[ModelAndParamRegression.SVR_Regression,ModelAndParamRegression.DecisionTree_Regressor,ModelAndParamRegression.RandomForest_Regressor,ModelAndParamRegression.GradientBoosting_Regressor,ModelAndParamRegression.KNeighbors_Regressor],permutate_n_less_column=0)
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

Total Permutations: 1
Total Permutations: 1
Total Permutations: 1
Total Permutations: 1


model_name                                                  Ridge_Regression
score                                                               0.235057
RMSE                                                         11237258.620821
total_columns                                                             29
scaler_type                                              QuantileTransformer
selected_features          cuisine, city, restaurant_location, facebook_p...
num_of_clusters_if_used                                                    5
accuracy                                                                None
f1_score                                                                None
Name: 1, dtype: object

In [31]:
data = ModelTrainingData(pre_processed_df, st.QUANTILE_TRANSFORMER, .90,use_pca=False,use_polynomials=False,use_feature_selection=True,create_clustering_feature_and_no_of_clusters=(True,5))

Executing RandomForestFeaturesSelection function to get the best features that are used in training the model.
Executing DecisionTreeClassifierFeaturesSelection function to get the best features that are used in training the model.
Executing LassoFeaturesSelection function to get the best features that are used in training the model.
Total columns :29
Selected number of columns: 23
Columns to use: facebook_popularity_quotient, live_music_rating, value_for_money, endorsed_by, order_wait_time, comfortablility, resturant_tier, privacy, fire_audit, overall_restaurant_rating, service, restaurant_zomato_rating, hygiene_rating, lively, liquor_license_obtained, restaurant_city_tier, dedicated_parking, cuisine, city, instagram_popularity_quotient, ambience, staff_responsivness, annual_turnover
Grid tuning in progress for KMeans...
Grid tuning in progress for Agglomerative Clustering...
Mean Shift clustering does not require hyperparameter tuning.
DBSCAN clustering does not require hyperparamete

In [32]:
trainer.data = data
trainer.perform_operation_regression(exclude_models=[ModelAndParamRegression.SVR_Regression,ModelAndParamRegression.DecisionTree_Regressor,ModelAndParamRegression.RandomForest_Regressor,ModelAndParamRegression.GradientBoosting_Regressor,ModelAndParamRegression.KNeighbors_Regressor],permutate_n_less_column=0)
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

Total Permutations: 1
Total Permutations: 1
Total Permutations: 1
Total Permutations: 1


model_name                                                  Ridge_Regression
score                                                               0.235057
RMSE                                                         11237258.620821
total_columns                                                             29
scaler_type                                              QuantileTransformer
selected_features          cuisine, city, restaurant_location, facebook_p...
num_of_clusters_if_used                                                    5
accuracy                                                                None
f1_score                                                                None
Name: 1, dtype: object

In [34]:
data = ModelTrainingData(pre_processed_df, st.QUANTILE_TRANSFORMER, .90,use_pca=True,use_polynomials=True,use_feature_selection=True,create_clustering_feature_and_no_of_clusters=(True,5))

Executing RandomForestFeaturesSelection function to get the best features that are used in training the model.
Executing DecisionTreeClassifierFeaturesSelection function to get the best features that are used in training the model.
Executing LassoFeaturesSelection function to get the best features that are used in training the model.
Total columns :29
Selected number of columns: 23
Columns to use: facebook_popularity_quotient, live_music_rating, value_for_money, endorsed_by, order_wait_time, comfortablility, resturant_tier, privacy, fire_audit, overall_restaurant_rating, service, restaurant_zomato_rating, hygiene_rating, lively, liquor_license_obtained, restaurant_city_tier, dedicated_parking, cuisine, city, instagram_popularity_quotient, ambience, staff_responsivness, annual_turnover
Grid tuning in progress for KMeans...
Grid tuning in progress for Agglomerative Clustering...
Mean Shift clustering does not require hyperparameter tuning.
DBSCAN clustering does not require hyperparamete

In [35]:
trainer.data = data
trainer.perform_operation_regression(exclude_models=[ModelAndParamRegression.SVR_Regression,ModelAndParamRegression.DecisionTree_Regressor,ModelAndParamRegression.RandomForest_Regressor,ModelAndParamRegression.GradientBoosting_Regressor,ModelAndParamRegression.KNeighbors_Regressor],permutate_n_less_column=0)
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

Total Permutations: 1
Total Permutations: 1
Total Permutations: 1
Total Permutations: 1


model_name                                                  Lasso_Regression
score                                                               0.239892
RMSE                                                         11201691.669553
total_columns                                                             22
scaler_type                                              QuantileTransformer
selected_features          facebook_popularity_quotient, live_music_ratin...
num_of_clusters_if_used                                                    5
accuracy                                                                None
f1_score                                                                None
Name: 10, dtype: object

In [37]:
data = ModelTrainingData(pre_processed_df, st.QUANTILE_TRANSFORMER, .95,use_pca=True,use_polynomials=True,use_feature_selection=True,create_clustering_feature_and_no_of_clusters=(True,5))

Executing RandomForestFeaturesSelection function to get the best features that are used in training the model.
Executing DecisionTreeClassifierFeaturesSelection function to get the best features that are used in training the model.
Executing LassoFeaturesSelection function to get the best features that are used in training the model.
Total columns :29
Selected number of columns: 23
Columns to use: facebook_popularity_quotient, live_music_rating, endorsed_by, value_for_money, order_wait_time, comfortablility, resturant_tier, privacy, fire_audit, overall_restaurant_rating, service, restaurant_zomato_rating, hygiene_rating, lively, liquor_license_obtained, restaurant_city_tier, dedicated_parking, cuisine, city, instagram_popularity_quotient, ambience, staff_responsivness, annual_turnover
Grid tuning in progress for KMeans...
Grid tuning in progress for Agglomerative Clustering...
Mean Shift clustering does not require hyperparameter tuning.
DBSCAN clustering does not require hyperparamete

In [38]:
trainer.data = data
trainer.perform_operation_regression(exclude_models=[ModelAndParamRegression.SVR_Regression,ModelAndParamRegression.DecisionTree_Regressor,ModelAndParamRegression.RandomForest_Regressor,ModelAndParamRegression.GradientBoosting_Regressor,ModelAndParamRegression.KNeighbors_Regressor],permutate_n_less_column=0)
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

Total Permutations: 1
Total Permutations: 1
Total Permutations: 1
Total Permutations: 1


model_name                                                  Ridge_Regression
score                                                               0.240189
RMSE                                                         11199503.206647
total_columns                                                             60
scaler_type                                              QuantileTransformer
selected_features          facebook_popularity_quotient, live_music_ratin...
num_of_clusters_if_used                                                    5
accuracy                                                                None
f1_score                                                                None
Name: 13, dtype: object

In [39]:
trainer.performance_df

,model_name,score,RMSE,total_columns,scaler_type,selected_features,num_of_clusters_if_used,accuracy,f1_score
0,Linear_Regression,0.234672,1.124009e+07,29,QuantileTransformer,"cuisine, city, restaurant_location, facebook_p...",5,None,None
1,Ridge_Regression,0.235057,1.123726e+07,29,QuantileTransformer,"cuisine, city, restaurant_location, facebook_p...",5,None,None
2,Lasso_Regression,0.234689,1.123996e+07,29,QuantileTransformer,"cuisine, city, restaurant_location, facebook_p...",5,None,None
3,ElasticNet_Regression,0.210525,1.141603e+07,29,QuantileTransformer,"cuisine, city, restaurant_location, facebook_p...",5,None,None
4,Linear_Regression,0.231358,1.126440e+07,23,QuantileTransformer,"facebook_popularity_quotient, live_music_ratin...",5,None,None
5,Ridge_Regression,0.231266,1.126507e+07,23,QuantileTransformer,"facebook_popularity_quotient, live_music_ratin...",5,None,None
6,Lasso_Regression,0.231352,1.126444e+07,23,QuantileTransformer,"facebook_popularity_quotient, live_music_ratin...",5,None,None
7,ElasticNet_Regression,0.207424,1.143843e+07,23,QuantileTransformer,"facebook_popularity_quotient, live_music_ratin...",5,None,None
8,Linear_Regression,0.239890,1.120170e+07,22,QuantileTransformer,"facebook_popularity_quotient, live_music_ratin...",5,None,None
9,Ridge_Regression,0.239882,1.120176e+07,22,QuantileTransformer,"facebook_popularity_quotient, live_music_ratin...",5,None,None


In [41]:
from auto_ml_kinder.classes.NeuralNetwork_BayesianOptimization_Params import NeuralNetwork_BayesianOptimization_Params

In [42]:
trainer.perform_neural_network_regression(totalExperiments=6,
                                          params=NeuralNetwork_BayesianOptimization_Params(
                                              neurons_min_max=(256,512)
                                              ,batch_size_min_max=(32,32)
                                              ,dropout_rate_min_max=(.3,.7)
                                              ,epochs_min_max=(60,70)
                                              ,hidden_layers_min_max=(3,6)  
                                              ,learning_rate_min_max=(.001,.1)
                                              ,normalization_min_max=(0,1)
                                              ,dropout_min_max=(1,1)
                                              ,activation_min_max=(0,3)
                                              ))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   | hidden... | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------
Neurons: 364||Total Columns:61||Normalization:False||Activation:selu||Total hidden layers:3||Dropout used:True||Dropout rate if used:0.6077049890092492||Optimizer:adadelta||Epochs:63||Batch_size:32

| 1         | -9.678e+1 | 1.837     | 32.0      | 1.0       | 0.6077    | 62.95     | 3.447     | 0.003225  | 363.6     | 0.2387    | 2.026     |
Neurons: 286||Total Columns:61||Normalization:False||Activation:elu||Total hidden layers:4||Dropout used:True||Dropout rate if used:0.567840095298652||Optimizer:nadam||Epochs:66||Batch_size:32

| 2         | -1.373e+1 | 2.972     | 32.0      | 1.0       | 0.5678    | 66.21     | 3.823     | 0.04716   | 286.3     | 0.07396   | 5.405     |
Neurons: 263||Total Col

In [43]:
trainer.neural_network_best_model(epochs=500)

AttributeError: 'ModelTrainer' object has no attribute 'neural_network_best_model'

In [ ]:
trainer.performance_df

In [ ]:
test = pd.read_csv('../data/Test_dataset.csv')

In [ ]:
test.columns = test.columns.str.lower().str.replace(' ', '_')

In [ ]:
new_column_name = label_encode[2].column_name
test = test.rename(columns={'endoresed_by': new_column_name})

In [ ]:
label_encode[0] = pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(test.city.value_counts().keys())}))

In [ ]:
model_config_test = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column=''
)

In [ ]:
test_processed = pp.process_test(test,model_config)

In [ ]:
trainer = mth.ModelTrainer(data=data)
trainer.perform_operation_regression(exclude_models=[ModelAndParamRegression.SVR_Regression,ModelAndParamRegression.DecisionTree_Regressor,ModelAndParamRegression.RandomForest_Regressor,ModelAndParamRegression.GradientBoosting_Regressor,ModelAndParamRegression.KNeighbors_Regressor],permutate_n_less_column=0)
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

In [ ]:
trainer.performance_df

In [ ]:
trainer.models

In [ ]:
predictions = trainer.predict_test_data(test_processed,trainer.models[1].model)

In [ ]:
test['predictions'] = predictions

In [ ]:
test